In [1]:
from Crypto.Cipher import AES, DES3
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from Crypto.Util import Counter
from base64 import b64decode, b64encode

# Triple DES

## Key generation

In [2]:
def generate_3des_key():
    while True:
        key = get_random_bytes(24)
        try:
            key = DES3.adjust_key_parity(key)
        except ValueError:
            continue
        
        k1, k2, k3 = key[:8], key[8:16], key[16:]
        
        if k1 != k2 and k2 != k3 and k1 != k3:
            return key, (k1, k2, k3)

key, (k1, k2, k3) = generate_3des_key()

print("3DES key (hex):", key.hex())
print("K1:", k1.hex())
print("K2:", k2.hex())
print("K3:", k3.hex())

3DES key (hex): 46d60dc8a7b3373470e340041c891f2f610b0da2ea982cf8
K1: 46d60dc8a7b33734
K2: 70e340041c891f2f
K3: 610b0da2ea982cf8


## Encryption

In [3]:
def encypt_3DES(plaintext, key):
    cipher = DES3.new(key, DES3.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(plaintext, DES3.block_size)) 
    initial_vector = b64encode(cipher.iv).decode('utf-8') #initialization vector -> probabilistic method
    ct = b64encode(ct_bytes).decode('utf-8') #cipher text
    return initial_vector, ct

data = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit,' \
'sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. ' \
'Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ' \
'ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate ' \
'velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, ' \
'sunt in culpa qui officia deserunt mollit anim id est laborum.'.encode('utf_8')
iv, ct = encypt_3DES(plaintext=data, key=key)
print(ct)

Sly9Npn+th/q/rfm7YhL8NscOIHLJWqaOoiQ4EVJyZj5Xx8NVCuUcEQ8gb7aHwN5/9vGx1n4YBvQdFOHnBil+6JeP1tGthtS5XYKyG7JSiwq3+aM7SidsdmIDcdogzVfKJbwNutkVtZtdudlYCr7rje7g0NjCNLAZTGwd8YHTFbXmzXXS1iq4Vo4VHXf/CX+CGOQR/Qrnz1OsrT7oD0kjC/73ZGxNJGw8aVvTNm7I58xNg81jHH9dhgrxTwCpYOJ7l4CwXJ0Ccu1qCDbvBGCrwBMhOTWQIZcSesaqp01reTZbQ8JWG58JnufzAYXBMIjjmskg7mKB+ClqahEDF3ff9rPtNIYGnUclZMVdmYTvaUbg5kZat6fr2hONdm1YUQzWPNAJ85TaDbcPP21HPaBdqQ2SjcwjpmBtoT/CtULxyIhNzLIO8nMFo5lS70KxvfLX3QJbu+9muyM87AYSSi+FWQwEszgy3QTP1O9beMgn5AqBMriLfnrWhOhis7nXmrClz6t7Y2xuD9rjAqbItcphTfLVsKyAXPZK7vkBD2QaaY+hCFJ2Y6I4xZMeLpIHDmIST4W2NmINt+0HVgsz/n6mQ==


## Decryption

In [4]:
def decrypt_3DES(iv, ct, key):
    try:
        cipher = DES3.new(key, DES3.MODE_CBC, iv= b64decode(iv))
        pt = unpad(cipher.decrypt(b64decode(ct)), DES3.block_size)
        print("The message was: ", pt)
    except (ValueError, KeyError):
        print("Incorrect decryption")

decrypt_3DES(iv, ct, key)

The message was:  b'Lorem ipsum dolor sit amet, consectetur adipiscing elit,sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'


# AES

## Encryption

In [5]:
nonce = get_random_bytes(8)    # 8-byte nonce (64 bits)
key = get_random_bytes(32)     # AES-256 key

ctr_encrypt = Counter.new(64, prefix=nonce)   # 64-bit counter, prefixed with nonce
aes_cipher_encrypt = AES.new(key, AES.MODE_CTR, counter=ctr_encrypt)
ciphertext = aes_cipher_encrypt.encrypt(data)

encoded_ciphertext = b64encode(ciphertext).decode("utf-8")
print("Ciphertext (Base64):", encoded_ciphertext)

Ciphertext (Base64): o7PHGAUK6dNPVAg35mBqJA9fGBkZMOGy7aAlVIPaWza1GPjqDkY4PSOAF3Zlyl1lzFVtfRXAd3TLUaEBAxC6H5bX8wY2lVgNc7oX5vaNCRZdLcQZx1tTQOvE1dxmUOf0Yy974snG1shZFA2tGz7LWH1mVq9YhOL0ylaSE83DzX439fCyfVjbdMPY8IaxNeKPvaMVxmE69ztLSUTZOGNWOXrn1DE3Un7zlB5Z9IbGydrEPD5H9OF17JpU+4iFHJ/2BYEgLPxlN5cQzGXt1wJ3ElLIAN2bIqeLRXw9013IDPI1iVPUXJqQRwzesBNQXVz257wSHXeq9j47uKzEWSSvxnNhja54WWR8MzndF7u37Igk5Ycwz9OsaqkX/jOSuNcr9wtDN/q0TJKmDL4EflHsNqszu+wmd9Jr6uDKe0ld2N5viIpG7h4tg6UXhxZ4aukgblyvRbZBQ221xHczioyeKHxi+EB7KMGUt7zcOHLdw4J7EVUvBOJyAa5Ca5G0RQuMNx8esswwOnVWehXPic3NbtwegNB0Ln/KZhYQQfLL867Qzam8svwQf6HzwKrbOC6kPPu1QPNFtOZhf/9/


## Decryption

In [6]:
ctr_dec = Counter.new(64, prefix=nonce)
aes_decipher = AES.new(key, AES.MODE_CTR, counter=ctr_dec)
decrypted_bytes = aes_decipher.decrypt(b64decode(encoded_ciphertext))

decrypted_text = decrypted_bytes.decode('utf-8')
print("Decrypted (readable):", decrypted_text)


Decrypted (readable): Lorem ipsum dolor sit amet, consectetur adipiscing elit,sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
